In [1]:
from nba_api.stats.endpoints import shotchartdetail
import json
import numpy as np
import requests
import pandas as pd
import seaborn as sns
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.patches import Circle, Rectangle, Arc, ConnectionPatch
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import mplcursors
import mpld3
import mpldatacursor


In [2]:
def shooting_stats(first, last, season):
    driver = webdriver.Chrome('/Users/patrickiskandar/Downloads/chromedriver')
    
    initial=last[0]
    last=last.replace("'", "")
    last_n=last[0:5]
    first_n=first[0:2]
    year=str((int(season[0:4])+1))
    #print(year)
    
    player_code=initial+'/'+last_n+first_n+'01'
    
    counter=0
    num=len(first)+len(last)+1
    URL="https://www.basketball-reference.com/players/"+player_code+"/shooting/"+year+"/"
    driver.get(URL.lower())
    name=driver.find_elements_by_xpath('//*[@id="meta"]/div/h1/span')[0].text
    
    while (counter==0):
        #print(name[0:num])
        if (name[0:num]).lower() == first.lower() + " " + last.lower():
            counter+=1
        else:
            suffix = '0'+(str((int(player_code[-1]))+1))
            #print(suffix)
            player_code=initial+'/'+last_n+first_n+suffix
            URL="https://www.basketball-reference.com/players/"+player_code+"/shooting/"+year+"/"
            driver.get(URL.lower())
            name=driver.find_elements_by_xpath('//*[@id="meta"]/div/h1/span')[0].text
            #print(name[0:num])
    
    players = driver.find_elements_by_xpath('//*[@id="shot-wrapper"]/div/div')
    #print(players)
    make = []
    miss=[]
    make_labels=[]
    miss_labels=[]
    for p in range(len(players)):
        if (players[p].get_attribute('class')=='tooltip make'):
            make.append(players[p].get_attribute('style'))
            make_labels.append(players[p].get_attribute('tip'))
        else:
            miss.append(players[p].get_attribute('style'))
            miss_labels.append(players[p].get_attribute('tip'))
    
    #print(labels[0])
    #print(players_list)
    make_LOC_X=[]
    make_LOC_Y=[]
    for i in range(len(make)):
        player_coord=make[i]
        digits = re.findall('\d+', player_coord)
        make_LOC_Y.append(int(digits[0])-50)
        make_LOC_X.append(int(digits[1])-240)
        #labels
        #print(digits)
    miss_LOC_X=[]
    miss_LOC_Y=[]
    for i in range(len(miss)):
        player_coord=miss[i]
        digits = re.findall('\d+', player_coord)
        miss_LOC_Y.append(int(digits[0])-50)
        miss_LOC_X.append(int(digits[1])-240)

    #print(LOC_X)
    #print(LOC_Y)

    df_make=pd.DataFrame()
    df_make['LOC_X']=make_LOC_X
    df_make['LOC_Y']=make_LOC_Y
    df_make['Labels']=make_labels
    
    df_miss=pd.DataFrame()
    df_miss['LOC_X']=miss_LOC_X
    df_miss['LOC_Y']=miss_LOC_Y
    df_miss['Labels']=miss_labels
    
    
    
    season2=str(int(season)+1)
    plt.title(first.capitalize()+" "+last.capitalize()+" "+season+"-"+season2+" "+"Shot Chart")
    #scatter=sns.scatterplot(x='LOC_X', y='LOC_Y', data=df, color='lightblue')
    concatenated = pd.concat([df_make.assign(dataset='makes'), df_miss.assign(dataset='misses')])
    sns.scatterplot(x='LOC_X', y='LOC_Y', data=concatenated, hue='dataset', palette=['green','red'], style='dataset', s=40)
    plt.show()
    driver.close()
#hi=df1.loc[df1['First']=='Shaquille'].values.flatten().tolist()
#print(hi)
#shooting_stats('Stephen', 'Curry', '2020')



In [3]:
from matplotlib.patches import Circle, Rectangle, Arc

def draw_court(ax=None, color='black', lw=2, outer_lines=False):
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = plt.gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 18" so it has a radius of 9", which is a value
    # 7.5 in our coordinate system
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-30, -7.5), 60, -1, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the side 3pt lines, they are 14ft long before they begin to arc
    corner_three_a = Rectangle((-220, -47.5), 0, 140, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((220, -47.5), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    # I just played around with the theta values until they lined up with the 
    # threes
    three_arc = Arc((0, 0), 475, 475, theta1=22, theta2=158, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)

    # List of the court elements to be plotted onto the axes
    court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                      bottom_free_throw, restricted, corner_three_a,
                      corner_three_b, three_arc, center_outer_arc,
                      center_inner_arc]

    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)

    return ax

In [ ]:
first = input("Enter player's first name: ")
last = input("Enter player's last name: ")
season = input("Enter first year of season: ")
plt.figure(figsize=(12,11))
draw_court(outer_lines=True, color="black")
plt.xlim(-300,300)
plt.ylim(0,470)
shooting_stats(first, last, season);
plt.show()